In [122]:
#!pip install konlpy
#!pip install sentence_transformers

In [123]:
# 필요한 모듈 불러오기
import numpy as np
import pandas as pd
import itertools
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [124]:
# 데이터 불러오기
from NewsAnalyzer.PreProcesser import preprocesser

file_path = "2022-04-12 news data, test.csv"

pc = preprocesser()
data = pc.road_data(file_path)

# 714번째 결측치 제거
# df = data.drop([714], axis = 0)
# df = df.reset_index()

data

,카테고리,언론사,기사제목,날짜,본문
0,정치,디지털타임스,"김준일 대표, ‘검수완박’ 민주당에 쓴소리…“‘논리적 확증편향’ 빠졌는지 돌아봐야”",2022.04.12.,\n\n\n\n\n‘계곡 살인’ 이은해 사건 거론하며 “‘검수완박’이었다면 이은해 ...
1,정치,중앙일보,[단독]윤희숙 父 ‘투기의혹’ 세종시 땅 매각…차익 전액 기부,2022.04.12.,\n\n\n\n\n 지난해 8월 25일 국민권익위원회 조사에서 부동산 관련 불법 ...
2,정치,문화일보,전쟁 피해 절박한 젤렌스키… 미지근한 한국 국회,2022.04.12.,"\n\n\n\n\n젤렌스키 우크라 대통령, “무기 요청” 국회 화상연설현장 참석 국..."
3,정치,아시아경제,"금태섭 ""文 정권 땐 특수부 키우더니 이젠 검수완박…이러니 내로남불""",2022.04.12.,"\n\n\n\n\n""응징적 차원에서 제도 바꾸면 중립성 해칠 것""금태섭 전 더불어민..."
4,정치,머니S,"송영길 ""애도 아니고 5선 정치인 진퇴가 가벼울 수 없다""",2022.04.12.,\n\n\n\n\n송영길 더불어민주당 전 대표가 12일 민주당의 지방선거 승리를 위...
...,...,...,...,...,...
1042,세계,한국일보,"친(親)러 도네츠크 군, “우크라 마리우폴 항 장악”",2022.04.12.,"\n\n\n\n\n우크라군, 항만시설ㆍ제철소에서 마지막 저항중 마리우폴 진격한 러시..."
1043,세계,YTN,러시아군 물러났지만...현실화된 우크라발 식량 위기,2022.04.12.,\n\n\n\n동영상 뉴스\n\n\n\n\n\t\n\t[앵커]러시아군이 수도 키이우...
1044,세계,뉴스1,"우크라 內 친러 도네츠크 군대 ""마리우폴 장악 임박""",2022.04.12.,\n\n\n\n\n우크라군 이미 몰려 있어…DPR군 전투 강화할 것마리우폴에 있는 ...
1045,세계,연합뉴스,"은혜 갚은 빈살만?…사우디 국부펀드, 쿠슈너에 2조5천억원 투자",2022.04.12.,\n\n\n\n\nNYT 보도…자문단 반대에도 경험 없는 쿠슈너 회사에 거액 투자2...


In [125]:
# 본문 전체를 돌리면 시간이 너무 오래 걸려서 테스트로 10개만 돌려봐았습니다.
content = data["본문"][:10]
content

0    \n\n\n\n\n‘계곡 살인’ 이은해 사건 거론하며 “‘검수완박’이었다면 이은해 ...
1    \n\n\n\n\n  지난해 8월 25일 국민권익위원회 조사에서 부동산 관련 불법 ...
2    \n\n\n\n\n젤렌스키 우크라 대통령, “무기 요청” 국회 화상연설현장 참석 국...
3    \n\n\n\n\n"응징적 차원에서 제도 바꾸면 중립성 해칠 것"금태섭 전 더불어민...
4    \n\n\n\n\n송영길 더불어민주당 전 대표가 12일 민주당의 지방선거 승리를 위...
5    \n\n\n\n\n검수완박 강행? 전국 檢 '직 걸고 막겠다' 보완수사 요청도 못해...
6    \n\n\n\n\n"검수완박 후속 조치에 의견 모여? 동의하기 힘들다""분명히 수사...
7    \n\n\n\n\n조국 전 법무부 장관 페이스북. /사진=뉴시스 [파이낸셜뉴스]  ...
8    \n\n\n\n\n"尹 신임 장관 자녀들 표창장 공개해야""허위면 학력 박탈하길"[...
9    \n\n\n\n\n김태흠, 라디오서 정치적 메시지보다 ‘명예 회복’ 우선 강조   ...
Name: 본문, dtype: object

In [126]:
# 불용어 제거

## 키워드 출력_test(Korean KeyBERT)

### 방법 1. 유사한 단어들 출력 - 문서와 가장 유사한 키워드 출력


In [127]:
# Okt 토큰화 - test는 이걸로 진행(토큰화, 불용어 제거까지 모두 되어 있다고 가정)

from konlpy.tag import Okt
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')

def keyBERT(content):
  keywords = []
  for i in range(len(content)):

    # 기사별 명사 추출
    okt = Okt()
    words = okt.nouns(content[i])

    # 추출한 명사 리스트에서 빼내기
    tokenized_nouns = ' '.join(word for word in words)

    n_gram_range = (1, 1)
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()

    # 모델 적용 
    doc_embedding = model.encode([tokenized_nouns])
    candidate_embeddings = model.encode(candidates)

    # 문서와 가장 유사한 키워드들을 추출
    top_n = 5
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    keyword = [candidates[index] for index in distances.argsort()[0][-top_n:]]
    keywords.append(keyword)
  return keywords

keyBERT(content)

[['노무현', '한국', '문재인', '김혜경', '김준일'],
 ['윤희', '서울', '윤모씨', '한국', '김경록'],
 ['응원', '침공', '기립박수', '런던', '영국'],
 ['형사소송법', '법무부', '출연', '문재인', '김현정'],
 ['서울시', '경기도', '출연', '인천', '김동연'],
 ['문재인', '김예원', '김후곤', '김현정', '김준일'],
 ['민생', '박홍근', '서울', '박지훈', '출연'],
 ['법무부', '고려대학교', '서울', '민씨', '부산대학교'],
 ['황교익', '황씨', '검찰총장', '민씨', '부산'],
 ['김태흠', '출연', '대한민국', '박정희', '충남도지사']]

## 방법 2. 다양한 키워드들을 얻기 위한 두 가지 알고리즘
### [1] Max Sum Similarity

In [128]:
def max_sum_sim(content, top_n, nr_candidates):
    keywords = []
    for i in range(len(content)):

      # 기사별 명사 추출
      okt = Okt()
      words = okt.nouns(content[i])

      # 추출한 명사 리스트에서 빼내기
      tokenized_nouns = ' '.join(word for word in words)

      n_gram_range = (1, 1)
      count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
      candidates = count.get_feature_names_out()

      # 모델 적용 
      doc_embedding = model.encode([tokenized_nouns])
      candidate_embeddings = model.encode(candidates)

      # 문서와 각 키워드들 간의 유사도
      distances = cosine_similarity(doc_embedding, candidate_embeddings)

      # 각 키워드들 간의 유사도
      distances_candidates = cosine_similarity(candidate_embeddings, candidate_embeddings)

      # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 pick.
      words_idx = list(distances.argsort()[0][-nr_candidates:])
      words_vals = [candidates[index] for index in words_idx]
      distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

      # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
      min_sim = np.inf
      candidate = None
      for combination in itertools.combinations(range(len(words_idx)), top_n):
          sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
          if sim < min_sim:
              candidate = combination
              min_sim = sim
      keyword = [words_vals[idx] for idx in candidate]
      keywords.append(keyword)
    return keywords



In [129]:
max_sum_sim(content, top_n=5, nr_candidates=30)

[['추구', '대립', '경찰', '노무현', '한국'],
 ['국민권익위', '문제', '부인', '서울', '김경록'],
 ['본회의', '대통령', '러시아', '박수', '런던'],
 ['청와대', '비판', '범죄수사', '활용', '김현정'],
 ['천해', '요구', '라디오', '박주민', '김동연'],
 ['카카오', '검찰', '법무부', '삼성', '서울'],
 ['검찰', '방향', '라디오', '청구권', '서울'],
 ['재판', '인사', '혐의', '장관', '서울'],
 ['검찰', '연합뉴스', '조국', '법무부', '민씨'],
 ['대화', '출마', '연합뉴스', '박근혜', '대한민국']]

In [130]:
max_sum_sim(content, top_n=5, nr_candidates=10)

[['경찰', '검찰', '노무현', '한국', '김혜경'],
 ['부친', '국민', '서울', '한국', '김경록'],
 ['박수', '국회의원', '침공', '기립박수', '런던'],
 ['법체계', '경찰', '형사소송법', '문재인', '김현정'],
 ['국민', '송영길', '박주민', '서울시', '김동연'],
 ['삼성', '박의', '서울', '대검찰청', '김현정'],
 ['국민', '영장', '서초동', '서울', '박지훈'],
 ['입학', '파이낸셜뉴스', '검찰', '법무부', '서울'],
 ['법무부', '윤잣대', '황교익', '민씨', '부산'],
 ['캡처', '대국민', '박근혜', '대한민국', '박정희']]

### [2] Maximal Marginal Relevance

In [131]:
def mmr(content, top_n, diversity):
    keywords = []
    for i in range(len(content)):

      # 기사별 명사 추출
      okt = Okt()
      words = okt.nouns(content[i])

      # 추출한 명사 리스트에서 빼내기
      tokenized_nouns = ' '.join(word for word in words)

      n_gram_range = (1, 1)
      count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
      candidates = count.get_feature_names_out()

      # 모델 적용 
      doc_embedding = model.encode([tokenized_nouns])
      candidate_embeddings = model.encode(candidates)

      # 문서와 각 키워드들 간의 유사도가 적혀있는 리스트
      word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)

      # 각 키워드들 간의 유사도
      word_similarity = cosine_similarity(candidate_embeddings)

      # 문서와 가장 높은 유사도를 가진 키워드의 인덱스를 추출.
      # 만약, 2번 문서가 가장 유사도가 높았다면
      # keywords_idx = [2]
      keywords_idx = [np.argmax(word_doc_similarity)]

      # 가장 높은 유사도를 가진 키워드의 인덱스를 제외한 문서의 인덱스들
      # 만약, 2번 문서가 가장 유사도가 높았다면
      # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 ... 중략 ...]
      candidates_idx = [i for i in range(len(candidates)) if i != keywords_idx[0]]

      # 최고의 키워드는 이미 추출했으므로 top_n-1번만큼 아래를 반복.
      # ex) top_n = 5라면, 아래의 loop는 4번 반복됨.
      for _ in range(top_n - 1):
        candidate_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis=1)

        # MMR을 계산
        mmr = (1-diversity) * candidate_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]

        # keywords & candidates를 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)
        keyword = [candidates[idx] for idx in keywords_idx]
      keywords.append(keyword)

    return keywords

In [132]:
mmr(content, top_n=5, diversity=0.2)

[['김준일', '김혜경', '문재인', '한국', '노무현'],
 ['김경록', '한국', '윤모씨', '서울', '윤희'],
 ['영국', '기립박수', '런던', '침공', '러시아'],
 ['김현정', '문재인', '출연', '형사소송법', '법무부'],
 ['김동연', '출연', '인천', '경기도', '서울시'],
 ['김준일', '김현정', '김후곤', '문재인', '김예원'],
 ['출연', '서울', '박지훈', '박홍근', '민생'],
 ['부산대학교', '민씨', '서울', '고려대학교', '검찰'],
 ['부산', '민씨', '검찰총장', '황교익', '법무부'],
 ['충남도지사', '박정희', '대한민국', '출연', '김태흠']]

In [133]:
mmr(content, top_n=5, diversity=0.7)

[['김준일', '정치권', '범죄자', '전세계', '침묵'],
 ['김경록', '경제학자', '기자회견', '국민권익위', '문제'],
 ['영국', '기립박수', '절박', '러시아', '공격'],
 ['김현정', '부패', '정책', '개혁', '페이스북'],
 ['김동연', '정치', '꿀벌', '지도', '라디오'],
 ['김준일', '정치검찰', '독일', '고통', '계획'],
 ['출연', '정치', '괴물', '폭탄', '민주당'],
 ['부산대학교', '칼럼니스트', '청원', '페이스북', '뇌출혈'],
 ['부산', '칼럼니스트', '청원', '학교생활기록부', '민주당'],
 ['충남도지사', '정치', '유영', '기자', '유튜브']]